<div align="center">
  <img src="https://raw.githubusercontent.com/lapalap/invert/main/assets/images/logo.svg" width="400"/>
</div>

<div align="center"><h1>Hello INVERT</h1>
<h5>Labeling Neural Representations with Inverse Recognition</h5>

## INVERT demo

Following notebook demonstrates the demo how INVERT works

In [1]:
! git clone https://github.com/lapalap/invert.git
! pip install git+file:///content/invert --quiet

Cloning into 'invert'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 301 (delta 0), reused 2 (delta 0), pack-reused 295
Receiving objects: 100% (301/301), 1.80 MiB | 4.88 MiB/s, done.
Resolving deltas: 100% (173/173), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 10.3 MB/s eta 0:00:00


In [2]:
from invert.explainer import Invert
from invert.metrics import Metric

import torch
import pandas as pd
import json

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen 

cuda:0


### 1.  Getting the data

In [3]:
# Downloading ResNet18 AvgPool layer activations across ImageNet validation dataset
! wget -O A50k_resnet18_avgpool.tnsr https://www.dropbox.com/scl/fi/9j6x953n7d504hgtn5eaa/A50k_avgpoolresnet18_val.tnsr?rlkey=tx9wz371p5m541799qzr18ybq&dl=0
# Downloading labels of ImageNet validation dataset
! wget -O ILSVRC2012_val.csv https://www.dropbox.com/scl/fi/d7silvnhen7mujts1h6by/ILSVRC2012_val.csv?rlkey=svo4g8f4wegx4ijo3pwfe3p6p&dl=0

--2023-11-27 18:00:28--  https://www.dropbox.com/scl/fi/9j6x953n7d504hgtn5eaa/A50k_avgpoolresnet18_val.tnsr?rlkey=tx9wz371p5m541799qzr18ybq
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf90202e9bc8d4c080393df3d5d.dl.dropboxusercontent.com/cd/0/inline/CIU4MfrmcmIdEgNhicT3qqyXTZSz9njUW48AveohgDZU9I9_zGg_1ak70mpUEkx-plc7QY1lSIg2WDqMq76JrFcdYD9WimsInUmNet6MpVAgRAWV57LS3_R8d2etqgdMP-FGI_kE9QhtnBum_TalBIMN/file# [following]
--2023-11-27 18:00:29--  https://ucf90202e9bc8d4c080393df3d5d.dl.dropboxusercontent.com/cd/0/inline/CIU4MfrmcmIdEgNhicT3qqyXTZSz9njUW48AveohgDZU9I9_zGg_1ak70mpUEkx-plc7QY1lSIg2WDqMq76JrFcdYD9WimsInUmNet6MpVAgRAWV57LS3_R8d2etqgdMP-FGI_kE9QhtnBum_TalBIMN/file
Resolving ucf90202e9bc8d4c080393df3d5d.dl.dropboxusercontent.com (ucf90202e9bc8d4c080393df3d5d.dl.dropboxusercontent.com)... 

#### 1.1 Take a look at the image labels

In [4]:
df = pd.read_csv('/content/ILSVRC2012_val.csv') #1481 class
one_hot_labels = torch.tensor(df[df.columns[1:]].values)

df

,image_name,n01440764,n01443537,n01484850,n01491361,n01494475,n01496331,n01498041,n01514668,n01514859,...,n09433442,n09613191,n00007846,n13135832,n11669921,n11665372,n11552386,n13083586,n00017222,n12992868
0,ILSVRC2012_val_00000001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ILSVRC2012_val_00000002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ILSVRC2012_val_00000003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ILSVRC2012_val_00000004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ILSVRC2012_val_00000005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,ILSVRC2012_val_00049996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,ILSVRC2012_val_00049997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,ILSVRC2012_val_00049998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,ILSVRC2012_val_00049999,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


#### 1.2 Investigating concepts

In [5]:
data = open('/content/invert/assets/imagenet/ILSVRC2012_classes.json')
data = data.read()
data = json.loads(data)

data['0']

{'name': 'tench.n.01',
 'definition': 'freshwater dace-like game fish of Europe and western Asia noted for ability to survive outside water',
 'offset': 'n01440764'}

### 2. Load activations to INVERT

In [6]:
explainer = Invert(
        storage_dir = ".invert/",
        device = device,
    )

A = torch.load("/content/A50k_resnet18_avgpool.tnsr")

explainer.load_activations(A,
                           one_hot_labels,
                           data,
                           dataset = "imagenet"
                           )

In [7]:
r = 296
L = 3
B = 3

explanation = explainer.explain_representation(r,
                                  L,
                                  B,
                                  Metric(),
                                  min_fraction=0.002,
                                  max_fraction=0.5,
                                  )

best_explanation = explanation[0]
best_explanation

{'formula': n02883344 & ~n03710193 & ~n03908618,
 'length': 3,
 'metric': tensor(0.9151, device='cuda:0'),
 'concept_fraction': tensor(0.0040, device='cuda:0')}

In [8]:
def find_by_offset(search_value:str, d:dict):
  for k in d:
    if d[k]['offset'] == search_value:
      return d[k]

In [11]:
print("Explanation for neuron {r}: ".format(r = r))
print(best_explanation['formula'])

print('\nHuman-understandable explanations: ')
for element in best_explanation['formula'].info["distinct_concepts"]:
  print(element, find_by_offset(str(element), data)['name'])

Explanation for neuron 296: 
n02883344 & ~n03710193 & ~n03908618

Human-understandable explanations: 
n03908618 pencil_box.n.01
n02883344 box.n.01
n03710193 mailbox.n.01
